
# Data Preprocessing with ADS - Diabetes Dataset

## Overview of this Notebook

`Data is the most important element in any data science project`.

Data Preparations for training any ML model involves Data Transformations and Manipulations. This is the most important phase and takes 80% of the time in the life cycle of any ML Model. The real world data is mostly incomplete, and has several missing values. Missing values can be because of the unavailability of data or inconsistency present in the data. There might be several errors and outliers present in the data. 
Preprocessing of data involves various steps - 

1. Combining attributes or columns  
2. Data Imputation
3. Data Cleaning
4. Dummy Variables Encoding
5. Outlier Detection
6. Feature Scaling
7. Feature Engineering
8. Feature Selection
9. Feature Extraction.

This notebook will demonstrate the core functionality of the Dataset Factory class. In this notebook, you will learn some of the many ways to clean and transform data in an `ADSDatasetFactory` Object.
When datasets are loaded with DatasetFactory, they can be transformed and manipulated easily with the built-in functions. Underlying, an ADSDataset object is a Pandas dataframe. Any operation that can be performed to a Pandas dataframe can also be applied to an ADS Dataset.

In [ ]:
import ads
import logging
import numpy as np
import os
import pandas as pd
import shutil
import tempfile
import warnings

from ads.dataset.dataset_browser import DatasetBrowser
from ads.dataset.factory import DatasetFactory
from ads.common.data import ADSData
from ads.common.model import ADSModel
from os import path
from sqlalchemy import create_engine
import seaborn as sns
from sklearn.base import TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import get_scorer
from collections import defaultdict
from ads.evaluations.evaluator import ADSEvaluator
#from ads.explanations.explainer import ADSExplainer

from ads.dataset.dataset_browser import DatasetBrowser
#from ads.feature_engineering import feature_type_manager, FeatureType
import matplotlib.pyplot as plt
from os import path

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

In [ ]:
print(ads.__version__)

In [ ]:
diab = pd.read_csv('diabetes_prediction_dataset.csv')

In [ ]:
diab_ds = DatasetFactory.from_dataframe(pd.DataFrame(diab), target = 'diabetes')

In [ ]:
type(diab_ds)

The ADSDataset object cannot be used for classification or regression problems `until a target has been set using set_target`.
This datasets is of type Binary Classification Dataset.

In [ ]:
diab_ds.head()

In [ ]:
diab_ds.shape

In [ ]:
diab_ds.summary()

### Remove Duplicate Rows

Having duplicate rows is a pain, as it will slow down your model training, without any actual gain. And therefore, we need to remove duplicates. We can call the `drop_duplicates` function to return a dataset with all of the duplicates removed.

In [ ]:
diab_ds.isna().sum()

In [ ]:
diab_without_dup = diab_ds.drop_duplicates()
diab_without_dup.shape


## show_in_notebook()
ADS offers a smart visualization tool that automatically detects the type of your data columns and offers the best way to plot your data. You can also create custom visualizations with ADS by using your preferred plotting libraries and packages. The correlation plot gives an idea on totally correlated pairwise attributes. `Highly correlated attributes contribute less to decision making`. So it is better to avoid those attributes. So the transformed dataset ensures that such attributes are avoided. The drop column feature drops those columns which are correlated. **`show_in_notebook()`** shows these correlations in the form of heatmaps. 
The ADS show_in_notebook() method creates a comprehensive preview of all the basic information about a dataset including:

a. The predictive data type (for example, regression, binary classification, or multinomial classification).

b. The number of columns and rows.

c. Feature type information.

d. Summary visualization of each feature.

e. The correlation map.

f. Any warnings about data conditions that you should be aware of.

To improve plotting performance, the ADS show_in_notebook() method uses an optimized subset of the data. This smart sample is selected so that it is statistically representative of the full dataset. The correlation map is only displayed when the data only has numerical (continuous or oridinal) columns.
The warnings in this case is the presence of null values for pregancies, insulin, skinthickness and outcome. This can be ignored because for this dataset this is not indicative of missing values.
The show_in_notebook() option shows 4 warnings. Pregnancies, SkinThicknes, Insulin, Outcome has zeros. Out of this, Outcome is the target variable. The other 3 attributes are just categorical which displays the presence or absence of Pregnancies, Insulin usage and SkinThickness. So this warning can be ignorned

In [ ]:
diab_ds.show_in_notebook()

## get_recommendations()
ADS provides built-in automatic transformation tools for datasets. These tools help detect issues with the data and recommend changes to improve the dataset. The recommended changes can be accepted by clicking a button in the drop-down menu. Once the changes are applied, the transformed dataset can be retrieved using the get_transformed_dataset() method.

To access the recommendations, you can use the get_recommendations() method on the ADSDataset object:

In [ ]:
diab_ds.get_recommendations()

## get_transformed_dataset()

Return the transformed dataset with the recommendations applied.

This method should be called after applying the recommendations using the recommendation `show_in_notebook()` API.

In [ ]:
diab_ds.get_transformed_dataset()

## Automated Transformations

Automatically chooses the most effective dataset transformation.

Alternatively, you can use auto_transform() to apply all the recommended transformations at once. auto_transform() returns a transformed dataset with several optimizations applied automatically. The optimizations include:

- Dropping constant and primary key columns, which has no predictive quality.

- Imputation to fill in missing values in noisy data.

- Dropping strongly co-correlated columns that tend to produce less generalizable models.

- Balancing a dataset using up or down sampling.

One optional argument to auto_transform() is `fix_imbalance`, which is set to `True` by default. When True, auto_transform() corrects any imbalance between the classes. ADS downsamples the dominant class first unless there are too few data points. In that case, ADS upsamples the minority class.

In [ ]:
diab_ds.auto_transform()

## select_best_features()

Automatically chooses the best features and removes the rest.

In [ ]:
diab_ds.select_best_features()